In [1]:
# https://towardsdatascience.com/pandas-groupby-a-simple-but-detailed-tutorial-314b8f37005d
# https://towardsdatascience.com/accessing-data-in-a-multiindex-dataframe-in-pandas-569e8767201d
# https://towardsdatascience.com/summarizing-data-with-pandas-crosstab-efc8b9abecf
# https://towardsdatascience.com/how-to-flatten-multiindex-columns-and-rows-in-pandas-f5406c50e569
# https://datascientyst.com/list-aggregation-functions-aggfunc-groupby-pandas/
# https://stackoverflow.com/questions/25929319/how-to-iterate-over-pandas-multiindex-dataframe-using-index
# https://stackoverflow.com/questions/24495695/pandas-get-unique-multiindex-level-values-by-label
# https://stackoverflow.com/questions/55706391/pandas-crosstab-on-multiple-columns-then-groupby

# https://matplotlib.org/stable/gallery/pyplots/pyplot_text.html#sphx-glr-gallery-pyplots-pyplot-text-py

In [2]:
import pandas as pd
import numpy as np
from myUtils import pickle_load, pickle_dump

path_dir = "C:/Users/ping/MyDrive/stocks/yfinance/"
path_data_dump = path_dir + "VSCode_dump/"

# # filename_symbols = path_data_dump + 'vg_symbols_4chars_max.csv'  # symbols text file
# filename_symbols = path_data_dump + 'my_symbols.csv'  # symbols text file

# _filename_pickled_df_OHLCVA_downloaded = 'df_OHLCVA_downloaded '  # OHLCVA downloaded from Yahoo
filename_pickled_df_adjOHLCV = 'df_adjOHLCV'  # adjusted OHLCV
filename_pickled_df_symbols_close = "df_symbols_close"  # symbols' adjusted close
filename_pickled_symbols_df_adjOHLCV =  'symbols_df_adjOHLCV'  # symbols in df_adjOHLCV
filename_pickled_perf_rank_dict =  'perf_rank_dict'  # store symbols from performance rank results
filename_pickled_r_all_ranks =  'r_all_ranks'  # list of top 100 most common symbols from performance rank results
filename_pickled_df_a = 'df_OHLCV_clean'  # df adjusted OHLCV, dropped symbols with no vol and close
filename_pickled_df_c = 'df_close_clean'  # df close, dropped symbols with no vol and close

verbose = False  # True prints more output

#################
# look_back_days = -250 * 60  # subset df iloc days
look_back_days = -250 * 6  # subset df iloc days, 6 years of data
#################

In [3]:
print(f"Full path to pickled df_symbols_close:  {path_data_dump}{filename_pickled_df_symbols_close}")
df_close = pickle_load(path_data_dump, filename_pickled_df_symbols_close, verbose=verbose)
print(f"Full path to pickled df_adjOHLCV:  {path_data_dump}{filename_pickled_df_adjOHLCV}")
df_adjOHLCV = pickle_load(path_data_dump, filename_pickled_df_adjOHLCV, verbose=verbose)

Full path to pickled df_symbols_close:  C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/df_symbols_close
Full path to pickled df_adjOHLCV:  C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/df_adjOHLCV


In [4]:
# https://stackoverflow.com/questions/63826291/pandas-series-find-column-by-value
df = df_adjOHLCV[look_back_days::]
df_v = df.xs('Volume', level=1, axis=1)  # select only Volume columns
rows, cols = np.where(df_v == 0)  # row index, column index where trading volumes are zero
idx_no_volume = list(set(cols))
idx_no_volume.sort()
symbols_no_volume = df_v.columns[idx_no_volume]
print(f'symbols with no volume:\n{symbols_no_volume}')

symbols with no volume:
Index(['AMCR', 'ATVI', 'AVT', 'BIIB', 'BKR', 'CCI', 'CHD', 'CNC', 'CNXC',
       'ESAB', 'FCFS', 'GEN', 'HZNP', 'IBKR', 'ICPT', 'JJC', 'NATI', 'PEAK',
       'PFG', 'SBNY', 'SRPT', 'SYNH', 'TWNK', 'VC', 'VNT', 'WH', 'XEL'],
      dtype='object')


In [5]:
df_dif = df_v - df_v.shift(periods=1)
rows, cols = np.where(df_dif == 0)
idx_same_volume = list(set(cols))
idx_same_volume.sort()
idx_same_volume
symbols_same_volume = df_v.columns[idx_same_volume]
print(f'symbols with same volume:\n{symbols_same_volume}')

symbols with same volume:
Index(['ABM', 'ACM', 'ACN', 'ADP', 'AGCO', 'ALRM', 'AMCR', 'AMED', 'AMG',
       'AMGN',
       ...
       'VRE', 'VRSK', 'VSAT', 'WERN', 'WEX', 'WSFS', 'WTS', 'WTW', 'XEL',
       'XENE'],
      dtype='object', length=244)


In [6]:
df_c = df.xs('Close', level=1, axis=1)  # select only Close columns
df_c = df_c.fillna(0).copy()  # convert NaNs to zeros
rows, cols = np.where(df_c == 0)  # row index, column index where trading volumes are zero
idx_no_close = list(set(cols))
idx_no_close.sort()
symbols_no_close = df_c.columns[idx_no_close]
print(f'symbols with NaN close:\n{symbols_no_close}')

symbols with NaN close:
Index(['ABNB', 'AIRC', 'ALGM', 'ATVI', 'BJ', 'BRBR', 'CARR', 'CDAY', 'CEG',
       'CHK', 'CHX', 'CNXC', 'CR', 'CRWD', 'CTVA', 'DDOG', 'DOW', 'DT', 'DTM',
       'ESAB', 'ETRN', 'FOX', 'FOXA', 'FYBR', 'GEHC', 'GFS', 'GO', 'GXO',
       'HZNP', 'ICPT', 'JJC', 'KD', 'LCID', 'MMP', 'MP', 'MRNA', 'NARI',
       'NATI', 'NCR', 'NVST', 'NVT', 'OFC', 'OGN', 'OSTK', 'OTIS', 'PDD',
       'PGNY', 'RIVN', 'SGOV', 'SHC', 'SITM', 'SWAV', 'SYNH', 'TWNK', 'VICI',
       'VNT', 'VSCO', 'WH', 'YETI', 'ZM', 'ZS'],
      dtype='object')


In [7]:
symbols_bad_data = list(symbols_no_close) + list(symbols_no_volume) + list(symbols_same_volume) # combine symbols with no volume and no close
unique_symbols_bad_data = sorted(list(set(symbols_bad_data)))  # unique symbols
print(f'unique symbols with bad data, e.g. no volume, same volume and $0 close, includes duplicate symbols: {len(unique_symbols_bad_data)}')

unique symbols with bad data, e.g. no volume, same volume and $0 close, includes duplicate symbols: 303


In [8]:
# get symbols of past model picks
df_picks = pickle_load(path_data_dump, 'df_picks', verbose=verbose)
df_picks.head(1)

,date_end_df_train,max_days_lookbacks,days_lookbacks,sym_freq_15,sym_freq_14,sym_freq_13,sym_freq_12,sym_freq_11,sym_freq_10,sym_freq_9,sym_freq_8,sym_freq_7,sym_freq_6,sym_freq_5,sym_freq_4,sym_freq_3,sym_freq_2
0,2023-11-09,120,"[30, 60, 120]",[],[],[],[],[],[],"['FTSM', 'SHV']",['CBOE'],"['GBTC', 'NRG']","['ANF', 'CAH', 'LRN']","['BTC-USD', 'EDU', 'GPS', 'USAC']","['PGR', 'TGH', 'YY']","['AGYS', 'ALL', 'CLS', 'HOLI', 'HRB']",[]


In [9]:
# keep only columns with symbols of past picks
df_picks.drop(columns=["date_end_df_train", "max_days_lookbacks", "days_lookbacks"], inplace=True)
df_picks.head(1)

,sym_freq_15,sym_freq_14,sym_freq_13,sym_freq_12,sym_freq_11,sym_freq_10,sym_freq_9,sym_freq_8,sym_freq_7,sym_freq_6,sym_freq_5,sym_freq_4,sym_freq_3,sym_freq_2
0,[],[],[],[],[],[],"['FTSM', 'SHV']",['CBOE'],"['GBTC', 'NRG']","['ANF', 'CAH', 'LRN']","['BTC-USD', 'EDU', 'GPS', 'USAC']","['PGR', 'TGH', 'YY']","['AGYS', 'ALL', 'CLS', 'HOLI', 'HRB']",[]


In [10]:
df_picks.columns

Index(['sym_freq_15', 'sym_freq_14', 'sym_freq_13', 'sym_freq_12',
       'sym_freq_11', 'sym_freq_10', 'sym_freq_9', 'sym_freq_8', 'sym_freq_7',
       'sym_freq_6', 'sym_freq_5', 'sym_freq_4', 'sym_freq_3', 'sym_freq_2'],
      dtype='object')

In [11]:
import ast

In [12]:
list_of_lists = []

for col in df_picks.columns:
  # convert column values from string to list, e.g. '[]', '[]', '["A", "B", ..]' ... to [], [], ["A", "B", ..], ...
  l_series = df_picks[col].apply(ast.literal_eval)
  # list_of_lists = [l_item for l_item in l_series if l_item]  # this doesn't works  
  for l_item in l_series:
      if l_item:  # 
        list_of_lists.append(l_item)  

symbols_picks = [val for sublist in list_of_lists for val in sublist]
print(f'symbol count from model picks: {len(symbols_picks)}')
# list sorted unique symbols
unique_symbols_picks = sorted(list(set(symbols_picks)))
print(f'unique symbol count from model picks: {len(unique_symbols_picks)}')

symbol count from model picks: 5800
unique symbol count from model picks: 294


In [13]:
def find_common_items(list1, list2):
  """Finds the common items between two lists.

  Args:
    list1: The first list.
    list2: The second list.

  Returns:
    A list of the common items between the two lists.
  """

  common_items = []
  for item in list1:
    if item in list2:
      common_items.append(item)
  return common_items


# code in if block runs only from command line, code will NOT be executed if imported as a module
if __name__ == "__main__": 
  list1 = ["a", "b", "c", "d", "e"]
  list2 = ["b", "c", "d", "f", "g"]
  common_items = find_common_items(list1, list2)
  print(common_items)

['b', 'c', 'd']


In [14]:
# symbols are in past picks but also have bad data 
common_symbols = find_common_items(unique_symbols_picks, unique_symbols_bad_data)
print(common_symbols)

['CHT', 'CMG', 'CVBF', 'CVLT', 'DVA', 'HOLI', 'HY', 'HZNP', 'ICPT', 'MLR', 'NATI', 'OSTK', 'PKX', 'SFBS', 'SRPT', 'TWNK', 'VNO', 'XENE']


In [15]:
def subtract_items(list1, list2):
  """Subtracts items in list2 from items in list1.

  Args:
    list1: The first list.
    list2: The second list.

  Returns:
    A list of the items in list1 that are not in list2.
  """

  subtracted_items = []
  for item in list1:
    if item not in list2:
      subtracted_items.append(item)
  return subtracted_items

In [16]:
symbols_drop = subtract_items(unique_symbols_bad_data, common_symbols)  # don't drop symbols in past picks
symbols_drop .sort()
print(f'len(unique_symbols_bad_data): {len(unique_symbols_bad_data)}')
print(f'len(common_symbols): {len(common_symbols)}')
print(f'len(symbols_drop): {len(symbols_drop)}')

len(unique_symbols_bad_data): 303
len(common_symbols): 18
len(symbols_drop): 285


In [17]:
# symbols_drop = list(symbols_no_close) + list(symbols_no_volume) + list(symbols_same_volume) # combine symbols with no volume and no close
# print(f'combined symbols with no volume, same volume and no close, inculdes duplicate symbols: {len(symbols_drop)}')
# symbols_drop = list(set(symbols_drop))  # drop duplicate symbols
# symbols_drop .sort()


df_a = df.drop(symbols_drop, axis=1, level=0)  # drop symbols from OHLCA df
df_c = df_close.iloc[look_back_days::]
df_c = df_c.drop(symbols_drop, axis=1)
print(f'unique symbols dropped from df_a (adjOLHLV) and df_c (Close): {len(symbols_drop)}')

unique symbols dropped from df_a (adjOLHLV) and df_c (Close): 285


In [18]:
print(f'symbols with no volume:   {len(symbols_no_volume):>5,}')
print(f'symbols with same volume: {len(symbols_same_volume):>5,}')
print(f'symbols with no close:    {len(symbols_no_close):>5,}\n')
print(f'symbols total before drop:                                        {len(df_close.columns):>5,}')
print(f'unique symbols dropped from df OHLCVA (df_a) and df Close (df_c): {len(symbols_drop):>5,}\n')
print('                                          symbols     rows')
print(f'df adjOHLCV (df_a) after dropped symbols:   {len(df_a.columns)/5:>5,.0f}    {len(df_a):>5,}')
print(f'df Close (df_c) after dropped symbols:      {len(df_c.columns):>5,}    {len(df_c):>5,}')


symbols with no volume:      27
symbols with same volume:   244
symbols with no close:       61

symbols total before drop:                                        1,561
unique symbols dropped from df OHLCVA (df_a) and df Close (df_c):   285

                                          symbols     rows
df adjOHLCV (df_a) after dropped symbols:   1,276    1,500
df Close (df_c) after dropped symbols:      1,276    1,500


In [19]:
pickle_dump(df_a, path_data_dump, filename_pickled_df_a)
print(f'pickled df adjOHLCV after dropping symbols with no volume, same volume, and no close:\n{path_data_dump}{filename_pickled_df_a}')
pickle_dump(df_c, path_data_dump, filename_pickled_df_c)
print(f'pickled df Close after dropping symbols with no volume, same volume, and no close:\n{path_data_dump}{filename_pickled_df_c}')

pickled df adjOHLCV after dropping symbols with no volume, same volume, and no close:
C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/df_OHLCV_clean
pickled df Close after dropping symbols with no volume, same volume, and no close:
C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/df_close_clean


In [20]:
from myUtils import list_dump

f_symbols_df_close_clean = 'symbols_df_close_clean.csv'  # symbols text file
symbols_df_c = list(df_c)  # column names in df_c
list_dump(symbols_df_c, path_data_dump, f_symbols_df_close_clean)# df_c.columns.to_csv(f_symbols_df_close_clean)

In [21]:
df_a.tail()

A                                                 \
                  Open        High         Low       Close     Volume   
Date                                                                    
2023-11-08  110.889999  111.570000  108.459999  109.389999  1495000.0   
2023-11-09  110.120003  110.120003  107.540001  107.739998  1481100.0   
2023-11-10  107.720001  108.480003  104.089996  108.470001  1847800.0   
2023-11-13  108.129997  108.269997  106.959999  107.459999  1200300.0   
2023-11-14  109.550003  112.290001  109.459999  111.610001  1971475.0   

                   AA                                              ...  \
                 Open       High        Low      Close     Volume  ...   
Date                                                               ...   
2023-11-08  25.120001  25.660000  24.965000  25.530001  3671400.0  ...   
2023-11-09  25.809999  25.870001  24.410000  24.580000  4078500.0  ...   
2023-11-10  24.700001  24.889999  24.280001  24.639999  3039400.0  ...   
2023-11-13  24.490000  25.025000  24.299999  24.600000  2839700.0  ...   
2023-11-14  25.440001  26.565001  25.360001  26.170000  5419367.0  ...   

                 ZUMZ                                               ZWS  \
                 Open       High    Low      Close    Volume       Open   
Date                                                                      
2023-11-08  16.500000  16.540001  15.83  15.910000  201200.0  28.379999   
2023-11-09  16.120001  16.219999  15.17  15.340000  240300.0  28.420000   
2023-11-10  15.370000  15.760000  14.99  15.740000  259500.0  28.280001   
2023-11-13  15.670000  15.940000  15.50  15.730000  115900.0  28.700001   
2023-11-14  16.270000  17.766001  16.27  17.709999  248992.0  29.360001   

                                                        
                 High        Low      Close     Volume  
Date                                                    
2023-11-08  28.379999  28.059999  28.299999  1163100.0  
2023-11-09  28.490000  28.049999  28.049999  1468500.0  
2023-11-10  28.860001  28.059999  28.799999  1413600.0  
2023-11-13  28.830000  28.389999  28.600000  1027700.0  
2023-11-14  30.030001  29.360001  29.950001  1213668.0  

[5 rows x 6380 columns]

In [22]:
df_c.tail()

,A,AA,AAL,AAP,AAPL,AB,ABBV,ABR,ABT,ACGL,...,YY,ZBH,ZBRA,ZD,ZG,ZION,ZTO,ZTS,ZUMZ,ZWS
Date,,,,,,,,,,,,,,,,,,,,,
2023-11-08,109.389999,25.530001,12.01,57.639999,182.649368,27.709999,142.029999,13.10,94.739998,84.120003,...,41.599998,104.860001,206.440002,60.990002,36.660000,33.759998,24.049999,170.869995,15.910000,28.299999
2023-11-09,107.739998,24.580000,11.75,56.139999,182.169998,27.160000,138.039993,12.86,93.550003,84.669998,...,41.180000,104.120003,202.520004,60.529999,35.189999,32.549999,23.690001,170.630005,15.340000,28.049999
2023-11-10,108.470001,24.639999,11.80,56.130001,186.399994,26.620001,138.589996,13.06,93.970001,85.949997,...,41.259998,105.419998,206.179993,60.750000,35.520000,32.919998,24.120001,169.320007,15.740000,28.799999
2023-11-13,107.459999,24.600000,11.78,55.939999,184.800003,26.080000,138.639999,12.98,95.790001,86.570000,...,41.799999,105.820000,202.610001,59.750000,34.529999,32.709999,24.020000,168.830002,15.730000,28.600000
2023-11-14,111.610001,26.170000,12.25,58.400002,187.440002,27.389999,138.059998,13.70,97.080002,85.500000,...,41.919998,108.070000,215.660004,66.019997,38.740002,35.369999,24.389999,172.649994,17.709999,29.950001
